In [4]:
# from resnet_18_fcn import *
from resnet_34_fcn import *
import time
from torch.utils.data import DataLoader
import torch
import gc
import voc
import torchvision.transforms as standard_transforms
import util
import numpy as np
import sys
import math
import copy

class MaskToTensor(object):
    def __call__(self, img):
        return torch.from_numpy(np.array(img, dtype=np.int32)).long()

# Initialize Weights with Xavier Weight Initialization
def init_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        torch.nn.init.xavier_uniform_(m.weight.data)
        torch.nn.init.normal_(m.bias.data) #xavier not applicable for biases
        if m.bias is not None:  # Check if the bias exists
            torch.nn.init.normal_(m.bias.data)  # Xavier not applicable for biases

In [5]:
mean_std = ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
input_transform = standard_transforms.Compose([
        standard_transforms.ToTensor(),
        standard_transforms.Normalize(*mean_std)
    ])
target_transform = MaskToTensor()

train_dataset =voc.VOC('train', transform=input_transform, target_transform=target_transform)
val_dataset = voc.VOC('val', transform=input_transform, target_transform=target_transform)
test_dataset = voc.VOC('test', transform=input_transform, target_transform=target_transform)

train_loader = DataLoader(dataset=train_dataset, batch_size= 16, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size= 16, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size= 16, shuffle=False)

epochs = 20
n_class = 21

fcn_model = FCN_ResNet34(n_class=n_class)
fcn_model.apply(init_weights)

C:\Users\hejin\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\hejin\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


AttributeError: 'NoneType' object has no attribute 'data'

In [3]:
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"

optimizer = torch.optim.Adam(fcn_model.parameters(), lr=5e-4)

# Choose an appropriate loss function from https://pytorch.org/docs/stable/_modules/torch/nn/modules/loss.html
criterion = nn.CrossEntropyLoss()
fcn_model = fcn_model.to(device)
max_model = fcn_model

earlystop = 3

In [4]:
def train():
    """
    Train a deep learning model using mini-batches.

    - Perform forward propagation in each epoch.
    - Compute loss and conduct backpropagation.
    - Update model weights.
    - Evaluate model on validation set for mIoU score.
    - Save model state if mIoU score improves.
    - Implement early stopping if necessary.

    Returns:
        None.
    """

    patience = 0
    best_iou_score = 0.0
    losses = []

    for epoch in range(epochs):
        ts = time.time()
        for iter, (inputs, labels) in enumerate(train_loader):
            
            optimizer.zero_grad() # reset optimizer gradients

            inputs, labels = inputs.to(device), labels.to(device) # both inputs and labels in device as model

            outputs = fcn_model(inputs) #  Compute outputs. Automatically in the same device as the model's

            loss = criterion(outputs, labels) #Calculate loss

            loss.backward() # Bckpropagate model

            optimizer.step() # Update the weights
            
            losses.append(loss.item())

            #if iter % 10 == 0:
            #    print("epoch{}, iter{}, loss: {}".format(epoch, iter, loss.item()))

        print("Finish epoch {}, time elapsed {}".format(epoch, time.time() - ts))
        print("Train Avg Loss: {}".format(np.mean(losses)))

        current_miou_score = val(epoch)

        # Save current IoU if better than stored best
        if current_miou_score > best_iou_score:
            best_iou_score = current_miou_score
            patience = 0
            max_model = copy.deepcopy(fcn_model) # save the best model
        else:
            patience += 1
            
        # Early stop if patience level is met
        if patience >= earlystop:
            print("Early stop at epoch " + str(epoch))
            break
    

In [5]:
def val(epoch):
    """
    Validate the deep learning model on a validation dataset.

    - Set model to evaluation mode. DONE
    - Disable gradient calculations. DONE
    - Iterate over validation data loader:
        - Perform forward pass to get outputs.
        - Compute loss and accumulate it.
        - Calculate and accumulate mean Intersection over Union (IoU) scores and pixel accuracy.
    - Print average loss, IoU, and pixel accuracy for the epoch.
    - Switch model back to training mode.

    Args:
        epoch (int): The current epoch number.

    Returns:
        tuple: Mean IoU score and mean loss for this validation epoch.
    """
    fcn_model.eval() # Put in eval mode (disables batchnorm/dropout) !
    
    losses = []
    mean_iou_scores = []
    accuracy = []
    
    with torch.no_grad(): # we don't need to calculate the gradient in the validation/testing

        # Iterate through Validation Set
        for iter, (input, label) in enumerate(val_loader):
            # label = (16, 224, 224) / batch size 16 of 244*244 masks
            # output = (16, 21, 224, 224) / batch size 16 of 21 possible classes of 244*244 masks
            
            input, label = input.to(device), label.to(device) # both inputs and labels in device as model
            
            output = fcn_model.forward(input) # Perform forward pass to get outputs.
            N, numClass, H, W = output.shape

            prediction = output.view(N, n_class, -1).argmax(dim=1).view(N, H, W) # Find the prediction for each pixel
            
            loss = criterion(output, label) # Compute loss and accumulate it.
            losses.append(loss.item())
            
            meanIOU = util.iou(prediction, label, n_class) # Calculate Intersection over Union (IoU) scores
            mean_iou_scores.append(meanIOU)

            acc = util.pixel_acc(prediction, label) # Calculate pixel accuracy
            accuracy.append(acc)
    
    print(f"Validation Loss: {np.mean(losses)}")
    print(f"Validation IoU: {np.mean(mean_iou_scores)}")
    print(f"Validation Pixel Acc: {np.mean(accuracy)}")
    print("\n")

    fcn_model.train() #TURNING THE TRAIN MODE BACK ON TO ENABLE BATCHNORM/DROPOUT!!

    return np.mean(mean_iou_scores)

def modelTest():
    """
    Test the deep learning model using a test dataset.

    - Load the model with the best weights.
    - Set the model to evaluation mode.
    - Iterate over the test data loader:
        - Perform forward pass and compute loss.
        - Accumulate loss, IoU scores, and pixel accuracy.
    - Print average loss, IoU, and pixel accuracy for the test data.
    - Switch model back to training mode.

    Returns:
        None. Outputs average test metrics to the console.
    """

    fcn_model = copy.deepcopy(max_model) # Asssume model loaded with the best weights.
    
    fcn_model.eval()  # Put in eval mode (disables batchnorm/dropout) !

    losses = []
    mean_iou_scores = []
    accuracy = []

    with torch.no_grad():  # we don't need to calculate the gradient in the validation/testing

        # Iterate through Test Set
        for iter, (input, label) in enumerate(test_loader):

            input, label = input.to(device), label.to(device) # both inputs and labels in device as model

            output = fcn_model.forward(input) # Perform forward pass to get outputs.
            N, numClass, H, W = output.shape

            prediction = output.view(N, n_class, -1).argmax(dim=1).view(N, H, W) # Find the prediction for each pixel
            
            loss = criterion(output, label) # Compute loss and accumulate it.
            losses.append(loss.item())
            
            meanIOU = util.iou(prediction, label, n_class) # Calculate Intersection over Union (IoU) scores
            mean_iou_scores.append(meanIOU)

            acc = util.pixel_acc(prediction, label) # Calculate pixel accuracy
            accuracy.append(acc)

    print(f"Test Loss at Test: {np.mean(losses)}")
    print(f"Test IoU at Test: {np.mean(mean_iou_scores)}")
    print(f"Test Pixel acc at Test: {np.mean(accuracy)}")

    fcn_model.train()  #TURNING THE TRAIN MODE BACK ON TO ENABLE BATCHNORM/DROPOUT!!

In [6]:
def exportModel(inputs):    
    """
    Export the output of the model for given inputs.

    - Set the model to evaluation mode.
    - Load the model with the best saved weights.
    - Perform a forward pass with the model to get output.
    - Switch model back to training mode.

    Args:
        inputs: Input data to the model.

    Returns:
        Output from the model for the given inputs.
    """

    fcn_model.eval() # Put in eval mode (disables batchnorm/dropout) !
    
    saved_model_path = "Fill Path To Best Model"
    # TODO Then Load your best model using saved_model_path
    
    inputs = inputs.to(device)
    
    output_image = fcn_model(inputs)
    
    fcn_model.train()  #TURNING THE TRAIN MODE BACK ON TO ENABLE BATCHNORM/DROPOUT!!
    
    return output_image

# if __name__ == "__main__":

#     val(0)  # show the accuracy before training
#     train()
#     modelTest()

#     # housekeeping
#     gc.collect()
#     torch.cuda.empty_cache()

In [19]:
train()

Finish epoch 0, time elapsed 10.932702541351318
Train Avg Loss: 2.2333646757262096
Validation Loss: 1.4381170315401894
Validation IoU: 0.053691666031312806
Validation Pixel Acc: 0.7392254264987247


Finish epoch 1, time elapsed 10.668454885482788
Train Avg Loss: 1.7796936631202698
Validation Loss: 1.3433957993984222
Validation IoU: 0.05372888506981775
Validation Pixel Acc: 0.7397263062591107


Finish epoch 2, time elapsed 10.195504665374756
Train Avg Loss: 1.5972679158051808
Validation Loss: 1.4163699405533927
Validation IoU: 0.05564781345682834
Validation Pixel Acc: 0.7508232094456085


Finish epoch 3, time elapsed 10.672451972961426
Train Avg Loss: 1.5055835034166063
Validation Loss: 1.2982743297304427
Validation IoU: 0.05564781345682834
Validation Pixel Acc: 0.7508232094456085


Finish epoch 4, time elapsed 10.23966932296753
Train Avg Loss: 1.4469209160123553
Validation Loss: 1.2275715725762504
Validation IoU: 0.05564781345682834
Validation Pixel Acc: 0.7508232094456085


Finish epo

In [21]:
modelTest()

Test Loss at Test: 1.4566880294254847
Test IoU at Test: 0.05575184499509766
Test Pixel acc at Test: 0.7289345076758381


## Q4.a COSINE ANNEALING LR

In [32]:
fcn_model = FCN_ResNet18(n_class=n_class)
fcn_model.apply(init_weights)
fcn_model = fcn_model.to(device)

earlystop = 3
max_model = fcn_model

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(fcn_model.parameters(), lr=1e-3)

#Test Cosine Annealing Learning Rate
iterMax = math.floor(len(train_dataset)/16)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=iterMax, eta_min=1e-4)

In [33]:
def train1():
    patience = 0
    best_iou_score = 0.0
    losses = []

    for epoch in range(epochs):
        ts = time.time()
        for iter, (inputs, labels) in enumerate(train_loader):
            
            optimizer.zero_grad() # reset optimizer gradients

            inputs, labels = inputs.to(device), labels.to(device) # both inputs and labels in device as model

            outputs = fcn_model(inputs) #  Compute outputs. Automatically in the same device as the model's

            loss = criterion(outputs, labels) #Calculate loss

            loss.backward() # Bckpropagate model

            optimizer.step() # Update the weights
            
            losses.append(loss.item())
            
            scheduler.step() # For cosine annealing learning rate

        print("Finish epoch {}, time elapsed {}".format(epoch, time.time() - ts))
        print("Train Avg Loss: {}".format(np.mean(losses)))
        
        current_miou_score = val(epoch)

        if current_miou_score > best_iou_score:
            best_iou_score = current_miou_score
            patience = 0
            max_model = copy.deepcopy(fcn_model)
            # save the best model
        else:
            patience += 1
            
        # Early stop if patience level is met
        if patience >= earlystop:
            print("Early stop at epoch " + str(epoch))
            break
    

In [27]:
train1()

Finish epoch 0, time elapsed 11.461528778076172
Train Avg Loss: 3.8488588503428867
Validation Loss: 3.8388543469565257
Validation IoU: 0.001378998427807684
Validation Pixel Acc: 0.012121172757607507


Finish epoch 1, time elapsed 12.173421144485474
Train Avg Loss: 3.656428813934326
Validation Loss: 3.3680985995701382
Validation IoU: 0.0013159682363144568
Validation Pixel Acc: 0.015353099945335277


Finish epoch 2, time elapsed 11.470860242843628
Train Avg Loss: 3.5253778468994867
Validation Loss: 3.1234386478151595
Validation IoU: 0.0019730341903034026
Validation Pixel Acc: 0.016004556817146503


Finish epoch 3, time elapsed 12.612285614013672
Train Avg Loss: 3.4190412844930376
Validation Loss: 2.9157194239752635
Validation IoU: 0.011553145074473066
Validation Pixel Acc: 0.15404490479227406


Finish epoch 4, time elapsed 12.382896423339844
Train Avg Loss: 3.3230171169553486
Validation Loss: 2.7621196508407593
Validation IoU: 0.014005449897335326
Validation Pixel Acc: 0.1831780169517583

In [28]:
modelTest()

Test Loss at Test: 1.4947880506515503
Test IoU at Test: 0.05722389361071704
Test Pixel acc at Test: 0.7301294087668548


## Q4.b Image Transformation

In [12]:
fcn_model = FCN_ResNet18(n_class=n_class)
fcn_model.apply(init_weights)
fcn_model = fcn_model.to(device)

earlystop = 3
max_model = fcn_model

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(fcn_model.parameters(), lr=1e-3)
#Test Cosine Annealing Learning Rate
iterMax = math.floor(len(train_dataset)/16)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=iterMax, eta_min=1e-4)

In [34]:
train1()

Finish epoch 0, time elapsed 11.864545345306396
Train Avg Loss: 2.752462693623134
Validation Loss: 2.881646156311035
Validation IoU: 0.0024142864631805333
Validation Pixel Acc: 0.0053823932614340386


Finish epoch 1, time elapsed 18.102809190750122
Train Avg Loss: 2.6775998898914883
Validation Loss: 2.5792002848216464
Validation IoU: 0.003260135881676691
Validation Pixel Acc: 0.020447400151466833


Finish epoch 2, time elapsed 29.49026584625244
Train Avg Loss: 2.600809403828212
Validation Loss: 2.345307230949402
Validation IoU: 0.013602761802062931
Validation Pixel Acc: 0.15890381927045374


Finish epoch 3, time elapsed 20.798587322235107
Train Avg Loss: 2.529697443757738
Validation Loss: 2.822207655225481
Validation IoU: 0.018248389034375367
Validation Pixel Acc: 0.1889300554903881


Finish epoch 4, time elapsed 14.17847466468811
Train Avg Loss: 2.464876183441707
Validation Loss: 2.2129252552986145
Validation IoU: 0.03271257134568092
Validation Pixel Acc: 0.40523533946223583


Finish 

In [14]:
modelTest()

Test Loss at Test: 1.3352243900299072
Test IoU at Test: 0.062609251938479
Test Pixel acc at Test: 0.727287003319743


## Q4.c Weight Imbalance + (Image Transformation)

In [45]:
def getClassWeights():
    ans = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    
    # Iterate through the training set
    for iter, (inputs, labels) in enumerate(train_loader):  
        unique_elements, counts = torch.unique(labels, return_counts=True)
        
        # Count number of each class
        for i in range(len(unique_elements)):
            ans[unique_elements[i]] += counts[i]

    normalized = [tensor.tolist() for tensor in ans]
    #normalized = [num/sum(normalized) for num in normalized]
    normalized = [1/(1-pow(0.1,num/50000)) for num in normalized]
    return torch.tensor(normalized)

In [46]:
class_weights = getClassWeights()
print(class_weights)
class_weights = class_weights.to(device)

tensor([1.0000, 1.1215, 1.1570, 1.0170, 1.1304, 1.0395, 1.0001, 1.0334, 1.0002,
        1.0061, 1.0280, 1.0023, 1.0014, 1.0195, 1.0275, 1.0000, 1.3059, 1.5267,
        1.0004, 1.0040, 1.0025])


In [51]:
fcn_model = FCN_ResNet18(n_class=n_class)
fcn_model.apply(init_weights)
fcn_model = fcn_model.to(device)

earlystop = 3
max_model = fcn_model

optimizer = torch.optim.Adam(fcn_model.parameters(), lr=5e-3)
criterion = nn.CrossEntropyLoss(weight=class_weights)
#Test Cosine Annealing Learning Rate
iterMax = math.floor(len(train_dataset)/16)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=iterMax, eta_min=1e-4)

In [52]:
train1()

Finish epoch 0, time elapsed 12.792518854141235
Train Avg Loss: 2.9731689180646623
Validation Loss: 3.3386610405785695
Validation IoU: 0.003711040629156048
Validation Pixel Acc: 0.013152827048788266


Finish epoch 1, time elapsed 13.60629391670227
Train Avg Loss: 2.8419202055249895
Validation Loss: 2.523624760763986
Validation IoU: 0.0008459458696763309
Validation Pixel Acc: 0.009729537741435859


Finish epoch 2, time elapsed 11.590258121490479
Train Avg Loss: 2.664101407641456
Validation Loss: 2.1430900607790266
Validation IoU: 0.04553097016372226
Validation Pixel Acc: 0.6137401881092839


Finish epoch 3, time elapsed 13.19899845123291
Train Avg Loss: 2.502972113234656
Validation Loss: 1.7477883441107613
Validation IoU: 0.049110519663740394
Validation Pixel Acc: 0.6178634954958546


Finish epoch 4, time elapsed 15.893030405044556
Train Avg Loss: 2.337155340399061
Validation Loss: 1.6107122216905867
Validation IoU: 0.05071620236024461
Validation Pixel Acc: 0.6155104211746083


Finish e

In [53]:
modelTest()

Test Loss at Test: 1.3887752549988883
Test IoU at Test: 0.057640198672544454
Test Pixel acc at Test: 0.7280713851528334
